In [1]:
import shutil
from pathlib import Path

import tensorflow as tf
import sle_gan

import yaml
with open("config.yaml", 'r') as stream:
    cfg = yaml.safe_load(stream)

print(cfg)

{'name': 'experiment', 'override': True, 'data_folder': 'data/dung_256x256_sle_gan_ready', 'resolution': 256, 'generator_weights': False, 'discriminator_weights': False, 'batch_size': 1, 'epochs': 1000, 'G_learning_rate': '2e-4', 'D_learning_rate': '2e-4', 'diff_augment': True, 'diff_augment_policies': 'translation,cutout', 'fid': False, 'fid_frequency': 1, 'fid_number_of_images': 128}


In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
_ = [tf.config.experimental.set_memory_growth(x, True) for x in physical_devices]

In [6]:
# Create folders for this experiment
# or override the old ones...

experiments_folder = Path("logs") / cfg["name"]

if experiments_folder.is_dir():
    if cfg["override"]:
        shutil.rmtree(experiments_folder)
    else:
        raise FileExistsError("Experiment already exists")
        
checkpoints_folder = experiments_folder / "checkpoints"
checkpoints_folder.mkdir(parents=True)

logs_folder = experiments_folder / "tensorboard_logs"
logs_folder.mkdir(parents=True)

In [7]:
#create the image dataset

dataset = sle_gan.create_dataset(batch_size=cfg["batch_size"], 
                                 folder=cfg["data_folder"], 
                                 resolution=cfg["resolution"],
                                 use_flip_augmentation=True, 
                                 shuffle_buffer_size=500)


In [3]:
# create Generator

G = sle_gan.Generator(cfg["resolution"])
sample_G_output = G.initialize(cfg["batch_size"])

# load G weights if they exist
if cfg["generator_weights"] is not False:
    G.load_weights(cfg["generator_weights"])
    print("Weights are loaded for G")
print(f"[Model G] output shape: {sample_G_output.shape}")

G_optimizer = tf.optimizers.Adam(learning_rate=cfg["G_learning_rate"])

2021-07-13 14:28:32.393054: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


[Model G] output shape: (1, 256, 256, 1)


In [4]:
# create Discriminator

D = sle_gan.Discriminator(cfg["resolution"])
sample_D_output = D.initialize(cfg["batch_size"])

if cfg["generator_weights"] is not False:
    D.load_weights(str(checkpoints_folder / "D_checkpoint.h5"))
    print("Weights are loaded for D")
print(f"[Model D] real_fake output shape: {sample_D_output[0].shape}")
print(f"[Model D] image output shape{sample_D_output[1].shape}")
print(f"[Model D] image part output shape{sample_D_output[2].shape}")

D_optimizer = tf.optimizers.Adam(learning_rate=cfg["D_learning_rate"])

[Model D] real_fake output shape: (1, 5, 5, 1)
[Model D] image output shape(1, 128, 128, 3)
[Model D] image part output shape(1, 128, 128, 3)


In [43]:
def read_image_from_path(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)
    return image

In [44]:
def create_dataset2(batch_size: int,
                   folder: str,
                   resolution: int,
                   use_flip_augmentation: bool = True,
                   image_extension: str = "jpg",
                   shuffle_buffer_size: int = 100):
    dataset = tf.data.Dataset.list_files(folder + f"/*.{image_extension}")
    dataset = dataset.map(read_image_from_path)
    dataset = dataset.map(partial(preprocess_images, resolution=resolution))
    dataset = dataset.shuffle(buffer_size=shuffle_buffer_size).batch(batch_size)
    
    #.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

In [45]:
def get_test_images2(batch_size: int, folder: str, resolution: int):
    dataset = create_dataset2(batch_size, str(folder), resolution=resolution, use_flip_augmentation=False,
                             shuffle_buffer_size=1)
    print(dataset.shape)
    
    #for x in dataset.take(1):
    #    return x

In [46]:
test_images = get_test_images2(2, cfg["data_folder"], cfg["resolution"])

NameError: name 'partial' is not defined

In [9]:
if cfg["fid"]:
    # Model for the FID calculation
    fid_inception_model = sle_gan.InceptionModel(height=cfg["resolution"], width=cfg["resolution"])

# what does this do?
test_input_size = 25
test_input_for_generation = sle_gan.create_input_noise(test_input_size)
test_images = sle_gan.get_test_images(test_input_size, cfg["data_folder"], cfg["resolution"])

# tb_file_writer = tf.summary.create_file_writer(str(logs_folder))
# tb_file_writer.set_as_default()

InvalidArgumentError: Cannot batch tensors with different shapes in component 0. First element had shape [256,256,1] and element 9 had shape [256,256,3]. [Op:IteratorGetNext]

In [ ]:
# set loss metrics

G_loss_metric = tf.keras.metrics.Mean()
D_loss_metric = tf.keras.metrics.Mean()
D_real_fake_loss_metric = tf.keras.metrics.Mean()
D_I_reconstruction_loss_metric = tf.keras.metrics.Mean()
D_I_part_reconstruction_loss_metric = tf.keras.metrics.Mean()

In [ ]:
# set augmentation policies

diff_augment_policies = None
if cfg["diff_augment"]:
    diff_augment_policies = cfg["diff_augment_policies"]

In [ ]:
# train gan

for epoch in range(cfg["epochs"]):
    print("### Epoch{epoch} ###".format(epoch))
    for step, image_batch in enumerate(dataset):
        G_loss, 
        D_loss, 
        D_real_fake_loss, 
        D_I_reconstruction_loss, 
        D_I_part_reconstruction_loss = sle_gan.train_step(
            G=G,
            D=D,
            G_optimizer=G_optimizer,
            D_optimizer=D_optimizer,
            images=image_batch,
            diff_augmenter_policies=diff_augment_policies)

        G_loss_metric(G_loss)
        D_loss_metric(D_loss)
        D_real_fake_loss_metric(D_real_fake_loss)
        D_I_reconstruction_loss_metric(D_I_reconstruction_loss)
        D_I_part_reconstruction_loss_metric(D_I_part_reconstruction_loss)

        if step % 100 == 0 and step != 0:
            print(f"\tStep {step} - "
                  f"G loss {G_loss_metric.result():.4f} | "
                  f"D loss {D_loss_metric.result():.4f} | "
                  f"D realfake loss {D_real_fake_loss_metric.result():.4f} | "
                  f"D I recon loss {D_I_reconstruction_loss_metric.result():.4f} | "
                  f"D I part recon loss {D_I_part_reconstruction_loss_metric.result():.4f}")

    if cfg["fid"]:
        if epoch % cfg["fid_frequency"] == 0:
            fid_score = sle_gan.evaluation_step(inception_model=fid_inception_model,
                                                dataset=dataset,
                                                G=G,
                                                batch_size=cfg["batch_size"],
                                                image_height=cfg["resolution"],
                                                image_width=cfg["resolution"],
                                                nb_of_images_to_use=cfg["fid_number_of_images"])
            print(f"[FID] {fid_score:.2f}")
            tf.summary.scalar("FID_score", fid_score, epoch)

    tf.summary.scalar("G_loss/G_loss", G_loss_metric.result(), epoch)
    tf.summary.scalar("D_loss/D_loss", D_loss_metric.result(), epoch)
    tf.summary.scalar("D_loss/D_real_fake_loss", D_real_fake_loss_metric.result(), epoch)
    tf.summary.scalar("D_loss/D_I_reconstruction_loss", D_I_reconstruction_loss_metric.result(), epoch)
    tf.summary.scalar("D_loss/D_I_part_reconstruction_loss", D_I_part_reconstruction_loss_metric.result(), epoch)

    print(f"Epoch {epoch} - "
          f"G loss {G_loss_metric.result():.4f} | "
          f"D loss {D_loss_metric.result():.4f} | "
          f"D realfake loss {D_real_fake_loss_metric.result():.4f} | "
          f"D I recon loss {D_I_reconstruction_loss_metric.result():.4f} | "
          f"D I part recon loss {D_I_part_reconstruction_loss_metric.result():.4f}")

    G_loss_metric.reset_states()
    D_loss_metric.reset_states()
    D_real_fake_loss_metric.reset_states()
    D_I_part_reconstruction_loss_metric.reset_states()
    D_I_reconstruction_loss_metric.reset_states()

    # TODO: save weights only when the FID score gets better
    G.save_weights(str(checkpoints_folder / "G_checkpoint.h5"))
    D.save_weights(str(checkpoints_folder / "D_checkpoint.h5"))

    # Generate test images
    generated_images = G(test_input_for_generation, training=False)
    generated_images = sle_gan.postprocess_images(generated_images, dtype=tf.uint8).numpy()
    sle_gan.visualize_images_on_grid_and_save(epoch, generated_images, experiments_folder / "generated_images",
                                              5, 5)

    # Generate reconstructions from Discriminator
    _, decoded_images, decoded_part_images = D(test_images, training=False)
    decoded_images = sle_gan.postprocess_images(decoded_images, dtype=tf.uint8).numpy()
    decoded_part_images = sle_gan.postprocess_images(decoded_part_images, dtype=tf.uint8).numpy()
    sle_gan.visualize_images_on_grid_and_save(epoch, decoded_images, experiments_folder / "reconstructed_whole_images",
                                              5, 5)
    sle_gan.visualize_images_on_grid_and_save(epoch, decoded_part_images,
                                              experiments_folder / "reconstructed_part_images", 5, 5)